# Numerical integration on time axis - Velocity

This example helps to clarify how to manage the time units in `pandas` and `scipy` to get a correct definite integral.

In [ ]:
It makes use of 

In [8]:
import pandas as pd
import numpy as np
from scipy import integrate

In [2]:
import io

In [26]:
data=io.StringIO('''
t,v
0,30
1,30
2,30
3,30
4,30
5,30
6,30
7,0
8,0
''')

In [27]:
df = pd.read_csv( data )
df

,t,v
0,0,30
1,1,30
2,2,30
3,3,30
4,4,30
5,5,30
6,6,30
7,7,0
8,8,0


In [35]:
df.index = pd.DatetimeIndex( start='2019-02-15  12:00:00', periods=9, freq='1H' )
df

,t,v
2019-02-15 12:00:00,0,30
2019-02-15 13:00:00,1,30
2019-02-15 14:00:00,2,30
2019-02-15 15:00:00,3,30
2019-02-15 16:00:00,4,30
2019-02-15 17:00:00,5,30
2019-02-15 18:00:00,6,30
2019-02-15 19:00:00,7,0
2019-02-15 20:00:00,8,0


In [33]:
integrate.trapz( df['v'].values/3600, x=df.index.astype(np.int64)/10**9 )

195.0

The correct result should be 210 km, because the velocities are in $km/h$

The error comes from the resolution of the measurements, and the trapezoid rule: There are 15km lost between the the hours 6 and 7.

If the velocities were taken in $m/s$ (but at the same times!), we wouldn't need to convert to seconds (the 3600 convertion factor).

In [36]:
integrate.trapz( df['v'].values, x=df.index.astype(np.int64)/10**9 )

702000.0

After 6 hours drive, we would have covered 702km. That makes sense, because $30 m/s$ are $108 km/h$, and after 6 hours we would be more than 600km away.

Now, if the velocity records were in miles per hour ($1 mile/h = 1.609 km/h) we would need to change only(!!) the hours to seconds. The result is then in miles.

In [37]:
integrate.trapz( df['v'].values/3600, x=df.index.astype(np.int64)/10**9 )

195.0

Those are 195 miles!

The rationale behind is that we are only telling the `integrate.trapz` function about the x-axis. It expects nanoseconds. Period. 

We take care of the units on the y-axis ourselves, because these units have no importance for the integration process.

Summarizing: Know the time units of the velocities, accelerations, flowing rates, growing rates, whatever you measure. Then change them to be _whatever-they-were-$/s$. And don't forget the `/10**9` to send nanoseconds in x, too.